In [16]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as io
import pandas as pd
from pandas import DataFrame as dframe
import cvxpy as cv

In [17]:
#対象被験者の特徴ベクトルをMATLABから入手
feat_C1 = np.loadtxt("feat_C1.txt",delimiter=",")
feat_C2 = np.loadtxt("feat_C2.txt",delimiter=",")
feat_Test_C1 = np.loadtxt("feat_Test_C1.txt",delimiter=",")
feat_Test_C2 = np.loadtxt("feat_Test_C2.txt",delimiter=",")

In [18]:
#非対象被験者の特徴ベクトルをMATLABから入手
feat_C1_K1 = np.loadtxt("feat_C1_K1.txt",delimiter=",")
feat_C2_K1 = np.loadtxt("feat_C2_K1.txt",delimiter=",")

In [19]:
#辞書行列を作る関数
def Dic(feat_l_tr,feat_r_tr): #入力の配列は2次元配列である必要がある
    feat_tr = np.concatenate([feat_l_tr, feat_r_tr],0)
    return feat_tr

In [20]:
#特徴ベクトルを2次元に変換
#特徴ベクトル g は２M×Tになっている必要がある
#Tは被験者の試行回数である
feat_tr = Dic(feat_C1,feat_C2)
#非対象被験者の特徴ベクトルも同様の処理を行う
feat_tr_K1 = Dic(feat_C1_K1,feat_C2_K1)
#それぞれの特徴ベクトルをつなげる
feat_tr_dic = Dic(feat_tr,feat_tr_K1).T
#feat_tr_dicはすべての特徴ベクトルをつなげたもの
#まず、分類に用いるテストデータの特徴ベクトルを2次元に変換
feat_l_te = feat_Test_C1;
feat_r_te = feat_Test_C2;
feat_l_te = feat_l_te.T
feat_r_te = feat_r_te.T
print(feat_l_te.shape,feat_tr_dic.shape,feat_tr.shape,feat_tr_K1.shape)
matrix_index = np.array([feat_C1.shape[0],feat_C2.shape[0],feat_C1_K1.shape[0],feat_C2_K1.shape[0]])

(4, 12) (4, 200) (120, 4) (80, 4)


In [21]:
#各クラスの特徴ベクトルの配列の列数を格納した配列を引数にすればもう少し拡張性が上がる
def SGRM(feat_C1,feat_C2,feat_tr,feat_tr_K1,feat_tr_dic,feat_C1_K1,feat_C2_K1,feat_l_te,matrix_index):
    λ_1 = 0.01
    λ_2 = 0.01
    answer = np.zeros(feat_l_te.shape[1])
    SGRM_Cla = np.zeros((feat_tr_dic.shape[1], feat_l_te.shape[1]))
    SGRM_l_Cla = np.zeros((feat_tr_dic.shape[1], feat_l_te.shape[1]))
    SGRM_r_Cla = np.zeros((feat_tr_dic.shape[1], feat_l_te.shape[1]))
    for i in range(feat_l_te.shape[1]):
        u = cv.Variable((feat_tr_dic.shape[1],1)) #目的関数の次元
        objective = cv.Minimize(0.5*(cv.norm(feat_tr_dic@u-feat_l_te[:,i,np.newaxis],2)**2)
                                +λ_1*cv.norm(u,1)
                                +λ_2*(cv.norm(u[0:feat_tr.shape[0],0],2)
                                      +cv.norm(u[feat_tr.shape[0]:feat_tr.shape[0]+feat_tr_K1.shape[0],0],2)))
        prob = cv.Problem(objective)
        result = prob.solve()
        #classVariableから値を取るためには.valueを付ける
        SGRM_Cla[:,i] = np.squeeze(u.value)
        #各クラスに関連する表現ベクトルの値を抜き出す
        #lは仕切りの値
        l = 0
        j = 0
        while j < matrix_index.shape[0]-1:
            SGRM_l_Cla[l:matrix_index[j],i] = SGRM_Cla[l:matrix_index[j],i]     
            l = l + matrix_index[j]
            j = j + 1
            SGRM_r_Cla[l:l+matrix_index[j],i] = SGRM_Cla[l:l+matrix_index[j],i]
            l = l + matrix_index[j]
            j = j + 1
    #パラメータを使って擬似的に各クラスのテスト信号の特徴ベクトルを作成
    dout_feat_l_te = np.zeros((feat_l_te.shape[0], feat_l_te.shape[1]))
    dout_feat_r_te = np.zeros((feat_l_te.shape[0], feat_l_te.shape[1])) 
    for i in range(feat_l_te.shape[1]):
        dout_feat_l_te[:,i] = feat_tr_dic@SGRM_l_Cla[:,i]
    for i in range(feat_r_te.shape[1]):
        dout_feat_r_te[:,i] = feat_tr_dic@SGRM_r_Cla[:,i]
    #作成した擬似特徴ベクトルとの残差を求める
    R_l =  np.zeros((feat_l_te.shape[0], feat_l_te.shape[1])) #残差を格納するための配列
    R_r =  np.zeros((feat_l_te.shape[0], feat_l_te.shape[1]))  
    for i in range(feat_l_te.shape[1]):
        R_l[:,i] = feat_l_te[:,i]-dout_feat_l_te[:,i]
        R_r[:,i] = feat_l_te[:,i]-dout_feat_r_te[:,i]
        if np.linalg.norm(R_l[:,i], ord=2) < np.linalg.norm(R_r[:,i], ord=2):
            answer[i] = 1
        elif np.linalg.norm(R_l[:,i], ord=2) > np.linalg.norm(R_r[:,i], ord=2):
            answer[i] = 2
    return answer

In [22]:
answer = SGRM(feat_C1,feat_C2,feat_tr,feat_tr_K1,feat_tr_dic,feat_C1_K1,feat_C2_K1,feat_l_te,matrix_index)
x = 0
for i in range(answer.shape[0]):
    if answer[i] == 1:
        x = x + 1
print((x/answer.shape[0])*100)

8.333333333333332
